In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

/Users/abdullahmarkus/VSProjects/llm-zoomcamp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/abdullahmarkus/VSProjects/llm-zoomcamp/.venv/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [8]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [4]:
df = df.iloc[:300]

In [5]:
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





# Q1

In [7]:
print(embedding_model.encode(df.iloc[0].answer_llm)[0])

-0.42244643


In [14]:
llm = embedding_model.encode("Das Haus ist schön")
true =embedding_model.encode("Das Haus ist schön")
import numpy as np

np.dot(llm, true.T)


38.9122

In [65]:
def sim_score(row, norm=False):
    row = row.copy()
    if norm:
        return row["emb_answer_llm_norm"].dot(row["emb_answer_orig_norm"]) 
    else:
        return row["emb_answer_llm"].dot(row["emb_answer_orig"])

def vec_norm(row):
    return row / np.sqrt((row * row).sum())
    

### Q2

In [47]:
df["emb_answer_llm"] = df["answer_llm"].apply(model.encode)
df["emb_answer_orig"] = df["answer_orig"].apply(model.encode)

In [68]:
print("Answer Q2:")
df["score"] = df.apply(sim_score,  norm=False, axis=1)

Answer Q2:


31.67430305480957

In [70]:
df["emb_answer_llm_norm"] = df["emb_answer_llm"].apply(vec_norm)
df["emb_answer_orig_norm"] = df["emb_answer_orig"].apply(vec_norm)
df["score_norm"] = df.apply(sim_score,  norm=True, axis=1)

# Q3

In [71]:
print("Answer Q3:")
df.score_norm.describe()["75%"]

Answer Q3:


0.8362346738576889

In [72]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [73]:
df.iloc[10]

answer_llm              Yes, all sessions are recorded, so if you miss...
answer_orig             Everything is recorded, so you won’t miss anyt...
document                                                         5170565b
question                             Are sessions recorded if I miss one?
course                                          machine-learning-zoomcamp
emb_answer_llm          [-0.10797268, -0.0706844, -0.091208324, 0.0922...
emb_answer_orig         [-0.22097449, -0.07662486, -0.19240245, -0.038...
score                                                           32.344715
emb_answer_llm_norm     [-0.01655762, -0.01083946, -0.013986805, 0.014...
emb_answer_orig_norm    [-0.034658495, -0.01201814, -0.030177144, -0.0...
score_norm                                                       0.777956
Name: 10, dtype: object

# Q4

In [92]:
from rouge import Rouge
rouge_scorer = Rouge()
data = df.iloc[10]

scores = rouge_scorer.get_scores(data['answer_llm'], data['answer_orig'])
scores[0]['rouge-1']['f']

0.45454544954545456

## Q5

In [87]:
f_scores = [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

# Calculate the mean of the 'f' scores
mean_f = sum(f_scores) / len(f_scores)

print(f"Mean 'f' score: {mean_f}")




Mean 'f' score: 0.35490034990035496


In [90]:
def get_rouge(row, measure, metric):
    return rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0][metric][measure]


df["rouge-2"] = df.apply(get_rouge,metric='rouge-2', measure='f', axis=1)
df["rouge-2"].describe()["mean"]
    

0.20696501983423318